In [4]:
!pip install deepface faiss-cpu numpy pillow

  Using cached deepface-0.0.93-py3-none-any.whl.metadata (30 kB)
     ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/88.4 kB ? eta -:--:--
     ------------- ------------------------ 30.7/88.4 kB 262.6 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 281.8 kB/s eta 0:00:01
     ----------------------------------- -- 81.9/88.4 kB 459.5 kB/s eta 0:00:01
     -------------------------------------- 88.4/88.4 kB 454.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/108.6 kB ? eta -:--:--
   ---------------------- ----------------- 61.4/108.6 kB 1.7 MB/s eta 0:00:01
   -------------------------- ------------- 71.7/108.6 kB 1.3 MB/s eta 0:00:01
   --------------------------------- ----- 92.2/108.6 kB 751.6 kB/s

In [5]:
import numpy as np
from deepface import DeepFace
import os

def generate_embeddings(image_path, model_name="Facenet"):
    """
    Generate embedding for a given image using DeepFace.
    :param image_path: Path to the image file
    :param model_name: Pre-trained model to use (Facenet, VGG-Face, etc.)
    :return: 128 or 512-dimensional embedding as a numpy array
    """
    try:
        embedding = DeepFace.represent(img_path=image_path, model_name=model_name)
        return np.array(embedding).astype(np.float32)  # Ensure float32 for FAISS compatibility
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def create_embedding_matrix(image_dir, model_name="Facenet"):
    """
    Generate embeddings for all images in a directory and return as a matrix.
    :param image_dir: Directory with all database images
    :param model_name: Pre-trained model to use (Facenet, VGG-Face, etc.)
    :return: (List of image paths, embeddings matrix)
    """
    embeddings = []
    image_paths = []
    
    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        if img_file.endswith(('jpg', 'jpeg', 'png')):
            embedding = generate_embeddings(img_path, model_name)
            if embedding is not None:
                embeddings.append(embedding)
                image_paths.append(img_path)
    
    return image_paths, np.array(embeddings)

# Example usage
image_dir = "C:\Users\rohan\Downloads\archive (6).zip\lfw-deepfunneled\lfw-deepfunneled"
image_paths, embeddings = create_embedding_matrix(image_dir, model_name="Facenet")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (991229426.py, line 40)

In [ ]:
import faiss

def create_faiss_index(embeddings):
    """
    Create a FAISS index for fast nearest neighbor search.
    :param embeddings: Matrix of embeddings (N, D)
    :return: FAISS index object
    """
    # Create a FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])  # Use L2 distance
    index.add(embeddings)  # Add embeddings to the index
    return index

# Create the FAISS index
faiss_index = create_faiss_index(embeddings)


In [ ]:
def search_similar_images(index, query_image_path, image_paths, model_name="Facenet", top_k=10):
    """
    Search for the top k similar images using FAISS.
    :param index: FAISS index object
    :param query_image_path: Path to the query image
    :param image_paths: List of database image paths
    :param model_name: Model to generate embedding for query image
    :param top_k: Number of top similar images to return
    :return: List of top k similar image paths with similarity scores
    """
    query_embedding = generate_embeddings(query_image_path, model_name)
    if query_embedding is None:
        return []

    # Search for the top_k nearest neighbors
    distances, neighbors = index.search(query_embedding.reshape(1, -1), top_k)
    top_k_results = [(image_paths[i], 1 - distances[0][j]) for j, i in enumerate(neighbors[0])]
    return top_k_results

# Example usage for searching
query_image = "C:\Users\rohan\OneDrive\Desktop\templates\rohan.jpg"
top_k_similar_images = search_similar_images(faiss_index, query_image, image_paths, model_name="Facenet", top_k=5)


In [ ]:
from PIL import Image

# Display results
print("Top 5 similar images:")
for img_path, similarity_score in top_k_similar_images:
    print(f"Image: {img_path}, Similarity Score: {similarity_score:.4f}")
    img = Image.open(img_path)
    img.show()
